In [ ]:
#Project files stored in git repository. Below is the link to the repository.
#https://github.com/TheXer0/ISY503-Project.git

In [55]:
#importing all required libraries

!pip3 install inflect
import json
import nltk
import os
import random
import re
import torch
import gensim
import string
import inflect
import torch.nn.functional as F
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from torch import nn, optim
from bs4 import BeautifulSoup
from IPython.display import display
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
p = inflect.engine()


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Xer0\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Xer0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Xer0\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xer0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Xer0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# removes non-English words from a string of text
#splits text into individual words, checks if each word is in a set of English words, 
#and then joins the remaining English words back together into a string
english_words = set(words.words())
def remove_non_english_words(text):
    words_list = text.split()
    filtered_words = [word for word in words_list if word.lower() in english_words]
    return ' '.join(filtered_words)

In [18]:
#cleans and filters a text input by removing punctuation and non-English words

def cleanReview(text):
  txt = text.translate(str.maketrans('','',string.punctuation))
  txt = remove_non_english_words(txt)
  return txt

def readFiles(filename):
    with open(filename,'r') as f:
        texto = f.read()  
    # Convert all text to lowercase
    texto = texto.lower()
    # Replace newline characters with empty string
    texto = texto.replace('\n', '')
    # Find all tags in the text and remove closing tags
    textohelp = re.sub(r'</.*?>', '', texto)
    # Extract names of all tags in the text
    etiquetas = re.findall(r'<.*?>', textohelp)
    nombres_etiquetas = [re.sub(r'[<>]', '', etiqueta) for etiqueta in etiquetas]
    # Find unique tags
    etiquetas_unicas = set(nombres_etiquetas)
    # Create a BeautifulSoup object from the text
    soup = BeautifulSoup(texto, 'html.parser')
    # Find all review elements and create a Python object
    reviews =soup.find_all('review')
    return  reviews

def loadReviews(revs,reviewType):
    #empty list
  reviewList = []
  for a in revs:
        #empty dictionaries dict
        dict = {}
        # Replace newline characters with empty string
        textoaux = a.prettify().replace('\n', '')
        if len(textoaux)>3000 or len(textoaux)<10 :
            continue
        try:
             # code that might raise an error if read a unknown character
            raiz = ET.fromstring(textoaux)
        except:
            continue
        # Mapping the new object
        if raiz.find('rating') is not None:
            dict['rating'] = raiz.find('rating').text
        if raiz.find('review_text') is not None:
            dict['review_text'] = cleanReview(raiz.find('review_text').text)
        dict['result']=reviewType
        
        reviewList.append(dict)
  return reviewList


# result = 0 NEGATIVE
raw_dataseta = readFiles('data/negative.review')
negRevs = loadReviews(raw_dataseta,0)

# result = 1 POSITIVE
raw_datasetb =  readFiles('data/positive.review')
posRevs = loadReviews(raw_datasetb,1)

# Full dataset NEG & POS reviews
data = negRevs + posRevs 
df = pd.DataFrame(data)
dataset = df.dropna()
dataset = dataset.copy()

print(len(dataset))
# Randomize the dataset
dataset = dataset.reindex(np.random.permutation(dataset.index))
dataset.head()

1828


,rating,review_text,result
1362,5.0,this book was a gift given to me by a friend a...,1
1340,5.0,you will either kill them or become a video on...,1
972,4.0,we would expect a well balanced approach to wr...,1
1682,4.0,or so my wife me she half the frozen section f...,1
468,1.0,why are there so many insanely positive for th...,0


In [19]:
# Remove repeating words
#takes in a list of sentences as input, 
#and uses the gensim library to preprocess each sentence by removing repeating words.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

In [20]:
#converts a list of words back into a sentence
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

#preprocesses text by splitting it into words, detokenizing the words, 
#and creating a list of preprocessed documents called data from a Pandas DataFrame column called review_text.

temp = []
#Splitting pd.Series to list
data_to_list = dataset['review_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append((data_to_list[i]))
data_words = list(sent_to_words(temp))
print(data_words[:5])
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

[['this', 'book', 'was', 'gift', 'given', 'to', 'me', 'by', 'friend', 'ago', 'own', 'and', 'have', 'read', 'about', 'twelve', 'on', 'the', 'topic', 'out', 'of', 'all', 'of', 'them', 'this', 'been', 'by', 'far', 'the', 'best', 'by', 'the', 'end', 'it', 'like', 'journey', 'inside', 'yourself', 'taken', 'place', 'and', 'you', 'arrive', 'to', 'new', 'destination', 'on', 'your', 'the', 'feel', 'of', 'the', 'book', 'is', 'one', 'of', 'love', 'and', 'support', 'not', 'the', 'type', 'or', 'author', 'as', 'guru', 'type', 'that', 'have', 'read', 'in', 'other', 'one', 'the', 'experience', 'of', 'two', 'professional', 'that', 'itself', 'in', 'each', 'chapter', 'it', 'made', 'large', 'difference', 'in', 'my', 'struggle', 'with', 'an', 'eating', 'disorder', 'that', 'at', 'age', 'am', 'now'], ['you', 'will', 'either', 'kill', 'them', 'or', 'become', 'video', 'on', 'television', 'that', 'for', 'modern', 'desensitization'], ['we', 'would', 'expect', 'well', 'balanced', 'approach', 'to', 'writing', 'abo

In [21]:
# function to clean the data and delete stop words
#returns a list of cleaned and lemmatized reviews

def text_procesing(text):
  STOPWORDS = set(stopwords.words('english'))
  cleaned_reviews = []
  for review in text:
    tokens = [word for word in word_tokenize(review) if not word in STOPWORDS]
    cleaned_reviews.append(" ".join(tokens))
  lemmatizer = WordNetLemmatizer()
  lem_reviews = []
  for review in cleaned_reviews:
    lem_reviews.append(" ".join(list(map(lemmatizer.lemmatize, word_tokenize(review)))))
  return lem_reviews

In [22]:
y = dataset['review_text']
dataset['review_text'] =text_procesing(y)

max_words = 5000
max_len = 200
# Create the token array and their sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataset['review_text'])
sequences = tokenizer.texts_to_sequences(dataset['review_text'])
reviews_sequences = sequence.pad_sequences(sequences, maxlen=max_len)
print(reviews_sequences)

[[   0    0    0 ...  671 2109  284]
 [   0    0    0 ... 1012 3296  272]
 [   0    0    0 ... 1151  465    7]
 ...
 [   0    0    0 ...  224 3516 4845]
 [   0    0    0 ...   89   90  619]
 [   0    0    0 ... 1190  522    1]]


In [23]:
# Y input variable sentiment categories 
result_sentiment = keras.utils.to_categorical(dataset['result'].astype('float32'), 2, dtype="float32")
print(result_sentiment)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [24]:
#Splitting the data
#The reviews_sequences and result_sentiment are the X and y input variables, respectively

X_train, X_test, y_train, y_test = train_test_split(reviews_sequences,result_sentiment, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

1371 457 1371 457


In [25]:
# Creation of the NN Sequential model using Keras Sequential API

fmodel = Sequential()
embedding_layer = Embedding(1000, 64)
fmodel.add(layers.Embedding(max_words, 15))
fmodel.add(layers.SimpleRNN(15,return_sequences=True))
fmodel.add(layers.SimpleRNN(15))
fmodel.add(layers.Dense(2,activation='softmax'))

#creates a sequential neural network model named 'model' and adds layers to it

model = Sequential()
embedding_layer = Embedding(1000, 64)
model.add(layers.Embedding(max_words, 15))
model.add(layers.SimpleRNN(15,return_sequences=True))
model.add(layers.SimpleRNN(15))
model.add(layers.Dense(2,activation='softmax'))

In [26]:
#compiles a model using the "rmsprop" optimizer and "categorical_crossentropy" loss function
#trains the model for 10 epochs
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
# history = fmodel.fit(X_train, y_train, epochs=10,callbacks=[PrintDot()])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Epoch 1/10
43/43 [==============================] - 3s 48ms/step - loss: 0.7004 - accuracy: 0.5004 - val_loss: 0.6927 - val_accuracy: 0.5186
Epoch 2/10
43/43 [==============================] - 2s 42ms/step - loss: 0.5785 - accuracy: 0.7520 - val_loss: 0.7126 - val_accuracy: 0.5295
Epoch 3/10
43/43 [==============================] - 2s 41ms/step - loss: 0.3441 - accuracy: 0.9015 - val_loss: 0.7779 - val_accuracy: 0.5164
Epoch 4/10
43/43 [==============================] - 2s 42ms/step - loss: 0.1696 - accuracy: 0.9657 - val_loss: 0.8831 - val_accuracy: 0.5230
Epoch 5/10
43/43 [==============================] - 2s 41ms/step - loss: 0.0818 - accuracy: 0.9825 - val_loss: 0.9713 - val_accuracy: 0.5033
Epoch 6/10
43/43 [==============================] - 2s 41ms/step - loss: 0.0401 - accuracy: 0.9934 - val_loss: 1.1176 - val_accuracy: 0.5383
Epoch 7/10
43/43 [==============================] - 2s 40ms/step - loss: 0.0193 - accuracy: 0.9985 - val_loss: 1.2361 - val_accuracy: 0.5164
Epoch 8/10
43

In [27]:
#fits the model to the training data for 10 epochs
fmodel.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
history = fmodel.fit(X_train, y_train, epochs=10)

Epoch 1/10
43/43 [==============================] - 3s 37ms/step - loss: 0.7055 - accuracy: 0.5237
Epoch 2/10
43/43 [==============================] - 2s 37ms/step - loss: 0.5318 - accuracy: 0.7338
Epoch 3/10
43/43 [==============================] - 2s 36ms/step - loss: 0.2986 - accuracy: 0.8862
Epoch 4/10
43/43 [==============================] - 2s 37ms/step - loss: 0.1483 - accuracy: 0.9504
Epoch 5/10
43/43 [==============================] - 2s 37ms/step - loss: 0.0663 - accuracy: 0.9781
Epoch 6/10
43/43 [==============================] - 2s 37ms/step - loss: 0.0361 - accuracy: 0.9898
Epoch 7/10
43/43 [==============================] - 2s 38ms/step - loss: 0.0147 - accuracy: 0.9964
Epoch 8/10
43/43 [==============================] - 2s 37ms/step - loss: 0.0114 - accuracy: 0.9964
Epoch 9/10
43/43 [==============================] - 2s 36ms/step - loss: 0.0044 - accuracy: 0.9985
Epoch 10/10
43/43 [==============================] - 2s 37ms/step - loss: 0.0039 - accuracy: 0.9985


In [28]:
# training set with validation data 
#fmodel.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#history = fmodel.fit(X_train, y_train, epochs=10,callbacks=[PrintDot()], validation_data=(X_test, y_test))

# training set with 20% of the training data as validation data
#fmodel.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#history = fmodel.fit(X_train, y_train, epochs=10, validation_split=0.2)

In [29]:
#creates a list of two strings, "Negative" and "Positive", 
#representing the two sentiment categories in the dataset

sentiment = ['Negative','Positive']

In [49]:
#prompts the user to input a review
review_input = input("WRITE THE SENTENCE AS AN INPUT HERE:")

WRITE THE SENTENCE AS AN INPUT HERE:The product makes good smothies and also is great for juices


In [51]:
#review_input= 'the product is horrible, and useless'
# review_input= 'this experience has been the worst , want all my money back'
# review_input='I never even traveled with it. It just stopped working completely making me lose data. NEVER AGAIN I'll buy anything from this company. This drive is a piece of rubbish!'

# POSITIVE Test reviews
# review_input = 'The product makes good smothies and also is great for juices'
# review_input= 'definitely recommend this memory card, download time is great'
# review_input= 'I borrowed this book on CD from the library and half way through, ordered the book so I would have it to refer to. Excellent book, highly recommended. Helps to put a realistic perspective on millionaires'



In [58]:
#uses the tokenizer to convert it to a sequence of tokens
tokensPrediction= tokenizer.texts_to_sequences([review_input])
sequencePrediction= sequence.pad_sequences(tokensPrediction, maxlen=max_len)

In [59]:
#generates a prediction of the sentiment based on a pre-trained model
# MODEL WITHOUT VALIDATION DATA

#test= sequence.pad_sequences(sequencer, maxlen=max_len)
#predictions =fmodel.predict(test) 
#print(predictions)
#sentiment[np.around(predictions, decimals=0).argmax(axis=1)[0]]

In [61]:
##generates a prediction of the sentiment based on trained RNN model
# MODEL WITH VALIDATION DATA
predictions =model.predict(sequencePrediction) 
print(predictions)
sentiment[np.around(predictions, decimals=0).argmax(axis=1)[0]]

1/1 [==============================] - 0s 221ms/step
[[0.08114872 0.9188513 ]]


'Positive'